In [2]:
import transformers
from transformers import BertConfig
config = BertConfig()
has_xla = hasattr(config,"xla_device")
print(has_xla)

False


In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: sjb15298306261 (sjb15298306261-Nanjing University). Use `wandb login --relogin` to force relogin


acc,▁▆▇▆██▇█
loss,▆█▅▂▃▃▁▂
acc,0.77075
loss,0.25984


In [3]:
import torch

# 检查CUDA是否可用
cuda_available = torch.cuda.is_available()

if cuda_available:
    # 获取GPU设备数量
    num_gpu = torch.cuda.device_count()

    # 获取当前使用的GPU索引
    current_gpu_index = torch.cuda.current_device()

    # 获取当前GPU的名称
    current_gpu_name = torch.cuda.get_device_name(current_gpu_index)

    # 获取GPU显存的总量和已使用量
    total_memory = torch.cuda.get_device_properties(current_gpu_index).total_memory / (1024 ** 3)  # 显存总量(GB)
    used_memory = torch.cuda.memory_allocated(current_gpu_index) / (1024 ** 3)  # 已使用显存(GB)
    free_memory = total_memory - used_memory  # 剩余显存(GB)

    print(f"CUDA可用，共有 {num_gpu} 个GPU设备可用。")
    print(f"当前使用的GPU设备索引：{current_gpu_index}")
    print(f"当前使用的GPU设备名称：{current_gpu_name}")
    print(f"GPU显存总量：{total_memory:.2f} GB")
    print(f"已使用的GPU显存：{used_memory:.2f} GB")
    print(f"剩余GPU显存：{free_memory:.2f} GB")
else:
    print("CUDA不可用。")

# 检查PyTorch版本
print(f"PyTorch版本：{torch.__version__}")

import torch
print(f"CUDA版本：{torch.version.cuda}")




CUDA可用，共有 1 个GPU设备可用。
当前使用的GPU设备索引：0
当前使用的GPU设备名称：NVIDIA GeForce RTX 3060 Laptop GPU
GPU显存总量：6.00 GB
已使用的GPU显存：0.00 GB
剩余GPU显存：6.00 GB
PyTorch版本：2.3.1+cu118
CUDA版本：11.8


In [2]:
# 必要的导入
import numpy as np
from typing import List

PHOSPHO_VOCAB = [
    "0",  # 未磷酸化
    "1",  # 磷酸化
]
SIGNALP_VOCAB = [
    "S",
    "I",
    "M",
    "O",
    "T",
    "L",
]
# 定义Tokenizer类
class Tokenizer:
    def __init__(self, labels: List[str] = SIGNALP_VOCAB):
        # build mapping
        # token_ids与字母表一一对应，即0-0,1-1
        token_ids = list(range(len(labels)))
        # 将上面的token_ids与标签建立映射
        self.vocab = dict(zip(labels, token_ids))

    def tokenize(self, text: str) -> List[str]:
        return [x for x in text]

    def convert_token_to_id(self, token: str) -> int:
        try:
            return self.vocab[token]
        except KeyError:
            raise KeyError(f"Unrecognized token: '{token}'")

    def convert_tokens_to_ids(self, tokens: List[str]) -> List[int]:
        return [self.convert_token_to_id(token) for token in tokens]
    
    def splitline_fasta(self, data):
        lines = data.split('\n')
        identifiers = lines[::3]
        # 第二行是蛋白质序列
        sequences = lines[1::3]
        # 第三行是标签
        labels = lines[2::3]
        return identifiers, sequences, labels
    
    def sequence_to_token_ids(self, sequence) -> List[int]:
        tokens = self.tokenize(labels)
        ids = self.convert_tokens_to_ids(tokens)
        return ids

# 创建Tokenizer实例
tokenizer = Tokenizer()

# 定义输入数据
item = "010110"  # 这是一个由0和1组成的字符串
testdata = """>P35583|EUKARYA|NO_SP|0
MLGAVKMEGHEPSDWSSYYAEPEGYSSVSNMNAGLGMNGMNTYMSMSAAAMGGGSGNMSAGSMNMSSYVG
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII"""
# 处理item，生成token_ids

test_tuple = tokenizer.splitline_fasta(testdata)

identifiers = test_tuple[0][0]  # 这里获取第一个标识符
sequences = test_tuple[1][0]    # 这里获取第一个序列
labels = test_tuple[2][0]       # 这里获取第一个标签

print("Identifiers:", identifiers)
print("Sequences:", sequences)
print("Labels:", labels)

token_ids = tokenizer.tokenize(labels)
print("Token_ids_split",token_ids)
token_ids = tokenizer.convert_tokens_to_ids(token_ids)

label_ids = tokenizer.sequence_to_token_ids(labels)

# 生成与token_ids相同形状的输入掩码，全为1
input_mask = np.ones_like(token_ids)

# 构建返回的元组
return_tuple = (
            np.array(token_ids),
            np.array(label_ids),
            np.array(input_mask),
        )

# 打印结果
print("Token IDs:", return_tuple[0])
print("label_ids:", return_tuple[1])
print("Input Mask:", return_tuple[2])



Identifiers: >P35583|EUKARYA|NO_SP|0
Sequences: MLGAVKMEGHEPSDWSSYYAEPEGYSSVSNMNAGLGMNGMNTYMSMSAAAMGGGSGNMSAGSMNMSSYVG
Labels: IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
Token_ids_split ['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']
Token IDs: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
label_ids: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Input Mask: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1